In [1]:
import json

from enbios.generic.mermaid2hierarchy import convert_mermaid_lines

In [2]:
mermaid1 = """graph BT
b1-->a
b2-->a"""

In [3]:
hierarchy = convert_mermaid_lines(mermaid1)
hierarchy

In [4]:
print(json.dumps(hierarchy, indent=2, ensure_ascii=False))

Specifying adapters and aggregators wtih `:::` (which can be done once per node) OR as link description 

`<node> -- <adapter_name> --> <parent_node>`

In [4]:
mermaid2 = """graph BT
b1-->a:::sum
b2:::bw-->a"""

hierarchy = convert_mermaid_lines(mermaid2)
hierarchy

In [5]:
mermaid2b = """
graph BT
    b1 --> a:::sum
    b2 -- bw --> a
"""

print(json.dumps(convert_mermaid_lines(mermaid2), indent=2, ensure_ascii=False))

In [7]:
mermaid2 = """graph BT
a:::sum
b1:::bw
b2:::bw
b1-->a
b2-->a"""

print(json.dumps(convert_mermaid_lines(mermaid2), indent=2, ensure_ascii=False))

In [6]:
mermaid3 = """graph BT
a:::sum
b1:::bw,code:xxx1
b2:::bw,code:xxx2

b1-->a
b2-->a"""

print(json.dumps(convert_mermaid_lines(mermaid3), indent=2, ensure_ascii=False))

## How to read a mermaid file convert it and add it to some experiment configuration

In [9]:
from enbios.generic.mermaid2hierarchy import convert_mermaid_file

hierarchy = convert_mermaid_file("data/mermaid/simple_mermaid.mermaid")

In [10]:
from enbios.base.experiment import Experiment

exp_config = {
    "adapters": [
        {
            "adapter_name": "brightway-adapter",
            "config": {"bw_project": "ecoinvent_391"},
            "methods": {
                "GWP1000": (
                    "ReCiPe 2016 v1.03, midpoint (H)",
                    "climate change",
                    "global warming potential (GWP1000)",
                ),
                "LOP": (
                    "ReCiPe 2016 v1.03, midpoint (E)",
                    "land use",
                    "agricultural land occupation (LOP)",
                ),
                "WCP": (
                    "ReCiPe 2016 v1.03, midpoint (E)",
                    "water use",
                    "water consumption potential (WCP)",
                ),
            },
        }
    ],
    "hierarchy": hierarchy,
}

exp = Experiment(exp_config)

In [11]:
exp.run()

We can also just pass the file location as a string to the hierarchy 

In [12]:
exp_config = {
    "adapters": [
        {
            "adapter_name": "brightway-adapter",
            "config": {"bw_project": "ecoinvent_391"},
            "methods": {
                "GWP1000": (
                    "ReCiPe 2016 v1.03, midpoint (H)",
                    "climate change",
                    "global warming potential (GWP1000)",
                ),
                "LOP": (
                    "ReCiPe 2016 v1.03, midpoint (E)",
                    "land use",
                    "agricultural land occupation (LOP)",
                ),
                "WCP": (
                    "ReCiPe 2016 v1.03, midpoint (E)",
                    "water use",
                    "water consumption potential (WCP)",
                ),
            },
        }
    ],
    "hierarchy": "data/mermaid/simple_mermaid.mermaid",
}

exp = Experiment(exp_config)
exp.run()

In [13]:
exp.result_to_dict()